# Working with Time-Series Data in a Consistent Bayesian Framework
---

Copyright 2017 Michael Pilosov

Demonstration available at https://www.youtube.com/watch?v=rUIVcl64NXw

### Import Libraries
_(should be 2.7 and 3.x compatible) _

In [1]:
# Interactivity
from ipywidgets import *
from cb_sandbox import *
%matplotlib inline

def make_tabulated_sandbox(num_experiments=1):
    # We create many copies of the same widget objects in order to isolate our experimental areas.
    num_samples = [widgets.IntSlider(value=1000, continuous_update=False, 
        orientation='vertical', disable=False,
        min=int(2.5E2), max=int(1E4), step=500, 
        description='Samples $N$') for k in range(num_experiments)]

    sd = [widgets.FloatSlider(value=0.25, continuous_update=False, 
        orientation='vertical', disable=False,
        min=0.05, max=1.75, step=0.05, 
        description='$\sigma$') for k in range(num_experiments)]

    lam_min, lam_max = 2.0, 7.0
    lam_bound = [widgets.FloatRangeSlider(value=[3.0, 6.0], continuous_update=False, 
        orientation='horizontal', disable=False,
        min=lam_min, max = lam_max, step=0.5, 
        description='Param: $\Lambda \in$') for k in range(num_experiments)]

    lam_0 = [widgets.FloatSlider(value=4.5, continuous_update=False, 
        orientation='horizontal', disable=False,
        min=lam_bound[k].value[0], max=lam_bound[k].value[1], step=0.1, 
        description='IC: $\lambda_0$') for k in range(num_experiments)]


    t_0 = [widgets.FloatSlider(value=0.5, continuous_update=False, 
        orientation='horizontal', disable=False,
        min=0.1, max=2.0, step=0.1,
        description='$t_0$ =', readout_format='.1f') for k in range(num_experiments)]

    Delta_t = [widgets.FloatSlider(value=0.1, continuous_update=False, 
        orientation='horizontal', disable=False,
        min=0.05, max=0.5, step=0.05,
        description='$\Delta_t$ =', readout_format='.2f') for k in range(num_experiments)]

    num_observations = [widgets.IntSlider(value=50, continuous_update=False, 
        orientation='horizontal', disable=False,
        min=1, max=100, 
        description='# Obs. =') for k in range(num_experiments)]
    
    compare = [widgets.Checkbox(value=False, disable=False,
        description='Observed v. Q(Post)') for k in range(num_experiments)]
    
    smooth_post = [widgets.Checkbox(value=True, disable=False,
        description='Smooth Posterior') for k in range(num_experiments)]
    
    fixed_noise = [widgets.Checkbox(value=True, disable=False,
        description='Fixed Noise Model') for k in range(num_experiments)]
    
    num_trials = [widgets.IntSlider(value=1, continuous_update=False, 
        orientation='vertical', disable=False,
        min=1, max=25, 
        description='Trials') for k in range(num_experiments)]
    
    Keys = [{'num_samples': num_samples[k], 
            'lam_bound': lam_bound[k], 
            'lam_0': lam_0[k], 
            't_0': t_0[k], 
            'Delta_t': Delta_t[k],
            'num_observations': num_observations[k], 
            'sd': sd[k],
            'compare': compare[k],
            'smooth_post': smooth_post[k],
            'fixed_noise': fixed_noise[k],
            'num_trials': num_trials[k]} for k in range(num_experiments)] 

    # Make all the interactive outputs for each tab and store them in a vector called out. (for output)
    out = [interactive_output(sandbox, Keys[k]) for k in range(num_experiments)]
    
    
    ### LINK WIDGETS TOGETHER (dependent variables) ###
    # if you change the bounds on the parameter space, update the bounds of lambda_0                          
    def update_lam_0(*args):
        k = tab_nest.selected_index
    #     lam_0[k].value = np.minimum(lam_0[k].value, lam_bound[k].value[1] )
    #     lam_0[k].value = np.maximum(lam_0[k].value, lam_bound[k].value[0] )
        lam_0[k].min = lam_bound[k].value[0] 
        lam_0[k].max = lam_bound[k].value[1]

    [lam_bound[k].observe(update_lam_0, 'value') for k in range(num_experiments)]


    ### GENERATE USER INTERFACE ###
    lbl = widgets.Label("UQ Sandbox", disabled=False)
    # horizontal and vertical sliders are grouped together, displayed in one horizontal box.
    # This HBox lives in a collapsable accordion below which the results are displayed.
    h_sliders = [widgets.VBox([lam_bound[k], lam_0[k], 
                               t_0[k], Delta_t[k], 
                               num_observations[k],
                              compare[k], smooth_post[k] ]) for k in range(num_experiments) ]
    v_sliders = [ widgets.VBox( [ fixed_noise[k], widgets.HBox([ num_samples[k], num_trials[k],
                               sd[k] ])] ) for k in range(num_experiments) ]
    user_interface = [widgets.HBox([h_sliders[k], v_sliders[k]]) for k in range(num_experiments)]
    
    # format the widgets layout (non-default options)
#     for k in range(num_experiments): 
#         h_sliders[k].layout.justify_content = 'center'
#         v_sliders[k].layout.justify_content = 'center'
#         options[k].layout.justify_content = 'center'
#         user_interface[k].layout.justify_content = 'center'

        
    ### MAKE TABULATED NOTEBOOK ###
    # Create our pages
    pages = [widgets.HBox() for k in range(num_experiments)]

    # instantiate notebook with tabs (accordions) representing experiments
    tab_nest = widgets.Tab()
    tab_nest.children = [pages[k] for k in range(num_experiments)]

    # title your notebooks
    experiment_names = ['Experiment %d'%(k+1) for k in range(num_experiments)]
    for k in range(num_experiments):
        tab_nest.set_title(k, experiment_names[k])

    # Spawn the children!!!
    for k in range(num_experiments):
    #     content = widgets.VBox( [user_interface[k], out[k]] )
        A = widgets.Accordion(children=[ user_interface[k] ])
        A.set_title(0,lbl.value)
        A.layout.justify_content = 'center'
        content = widgets.VBox([ A, out[k]  ])
        content.layout.justify_content = 'center'
        tab_nest.children[k].children = [content]
    
    return tab_nest, Keys

# define wrapper function to repeat trials. 

---
## Defining the Parameter to Observables (PtO) and Quantity of Interest (QoI) maps

---
Consider the Initival Value Problem (IVP)

$$
\begin{cases}
    \dot{u}(t) = -u(t), & t>0 \\
    u(0) = \lambda, & 
\end{cases}
$$

with solution $u(t;\lambda) = \lambda \,e^{-t}$.

Let $0<t_0<t_1<\ldots, t_K$ denote the observation times. 
Given a fixed initial condition (i.e., parameter) $\lambda$, let $y_k$ denote the set of (noisy) observations of the state variable $u(t_k,\lambda)$ for $k=0,1,\ldots, K$. 

We make the standard assumption of an additive error model with independent identically distributed noise, i.e., for each $k=0,1,\ldots,K$ and fixed value of $\lambda$, we assume that the Parameter-to-Observables (PtO) maps are given by

$$
O_k(\lambda) := u(t_k;\lambda) + \epsilon_k, \quad \epsilon_k \sim N(0,\sigma_k). 
$$

Assume that there is a true value of $\lambda$, which we denoted by $\lambda_0$, for which the observations $y_k:=O_k(\lambda_0)$ are given for $k=0,1,\ldots,K$.

Then, for any other value of $\lambda$ in the IVP above, we define the Quantity of Interest (QoI) as the **Weighted Sum Squared Error (a weighted 2-norm) between the observations and the model predictions**, i.e., we define the QoI map as

$$
    \boxed{Q(\lambda) := \sum_{k=0}^{K} \frac{(u(t_k;\lambda) - y_k) ^ 2}{\sigma_k^2}}
$$

We let $\mathcal{D} := Q(\Lambda)$ denote the space of all possible observations of mean squared error. 


---
## Formulating the Inverse Problem:
---
### Prior Information/Assumptions

* We assume that the true value $\lambda_0$ belongs to the parameter space defined by $\Lambda:= [0, 2]$.


* Prior to the data $\{y_k\}_{k=0}^K$ being available, any value of the parameter $\lambda$ in $\Lambda$ is assumed to be equally likely. In other words, we take $\pi^{prior}_\Lambda(\lambda)$ to be a uniform density.


### The Observed Density

* For the true value of $\lambda_0$, we have that $u(t_k;\lambda_0)-y_k = \epsilon_k$ for each $k$. Thus, the observed density on $\mathcal{D}$, denoted by $\pi^{obs}_{\mathcal{D}}(d)$, is given by a $\chi^2_{K+1}$ distribution.

### The Posterior Density

* Let $\pi^{O(prior)}_{\mathcal{D}}(d)$ denote the push-forward of the prior density onto $\mathcal{D}$. Then, the posterior density on $\Lambda$ is given by

$$
    \pi^{post}_\Lambda(\lambda) := \pi^{prior}_\Lambda(\lambda)\frac{\pi^{obs}_{\mathcal{D}}(Q(\lambda))}{\pi^{O(prior)}_{\mathcal{D}}(Q(\lambda))}
$$

---
## The numerical implementation and practical considerations
---
Here, we provide only a few brief remarks on the implementation.
For a step-by-step walkthrough, please see the CBayes_TS.ipynb file.
Below you will find an all-in-one version. 

***Some useful remarks go here.***

* In the `sandbox` function below, `T` is an interval of observation times



---

### Define some functions for the sandbox

---

# All-in-One Sandbox!
_Run the cells below to start experimenting_

In [2]:
# Display the "tabulated nest" with this many tabs
num_experiments = 6

### Set up an example to illustrate some observations of this system

The follow are available handles:  
`num_samples`  `sd`  `num_trials`  
`lam_bound`  `lam_0`    
`t_0`  `Delta_t`  `num_observations`  

`compare`  `smooth_post`  `fixed_noise`


### Measuring Earlier is Advantageous

In [3]:
# acc = tab_nest.children[1].children[0].children[0] # accordion object. how to collapse??

In [4]:
def set_prop(K, prop, val):
    for k in range(num_experiments): # for each tab
        K[k][prop].value = val

In [5]:
times = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6]

T1, K1 = make_tabulated_sandbox(num_experiments)
for k in range(num_experiments):
    K1[k]['t_0'].value = times[k]
    K1[k]['num_trials'].value = 5
    K1[k]['sd'].value = 0.15

prop = 'fixed_noise'
interact(set_prop, K = fixed(K1), prop=fixed(prop), 
                val = Checkbox(value=True, description = 'Fix Noise') );
prop = 'smooth_post'
interact(set_prop, K = fixed(K1), prop=fixed(prop), 
                val = Checkbox(value=True, description = 'Smooth Post') );

prop = 'num_observations'
interact_manual(set_prop, K = fixed(K1), prop=fixed(prop), 
                val = IntSlider(value=50, min=1, max=100, description='Samp. $N$ =') );
prop = 'Delta_t'
interact_manual(set_prop, K = fixed(K1), prop=fixed(prop), 
                val = FloatSlider(value=0.1, min=0.05, max=0.5, step=0.05, description='$\Delta_t$ =') );
prop = 'sd'
interact_manual(set_prop, K = fixed(K1), prop=fixed(prop), 
                val = FloatSlider(value=0.1, min=0.05, max=0.25, step=0.01, description='Constant $\sigma$:' ));

T1

interactive(children=(Checkbox(value=True, description='Fix Noise'), Output()), _dom_classes=('widget-interact',))

interactive(children=(Checkbox(value=True, description='Smooth Post'), Output()), _dom_classes=('widget-interact',))

interactive(children=(IntSlider(value=50, description='Samp. $N$ =', min=1), Button(description='Run Interact', style=ButtonStyle()), Output()), _dom_classes=('widget-interact',))

interactive(children=(FloatSlider(value=0.1, description='$\\Delta_t$ =', max=0.5, min=0.05, step=0.05), Button(description='Run Interact', style=ButtonStyle()), Output()), _dom_classes=('widget-interact',))

interactive(children=(FloatSlider(value=0.1, description='Constant $\\sigma$:', max=0.25, min=0.05, step=0.01), Button(description='Run Interact', style=ButtonStyle()), Output()), _dom_classes=('widget-interact',))

Tab(children=(HBox(children=(VBox(children=(Accordion(children=(HBox(children=(VBox(children=(FloatRangeSlider(value=(3.0, 6.0), continuous_update=False, description='Param: $\\Lambda \\in$', max=7.0, min=2.0, step=0.5), FloatSlider(value=4.5, continuous_update=False, description='IC: $\\lambda_0$', max=6.0, min=3.0), FloatSlider(value=0.1, continuous_update=False, description='$t_0$ =', max=2.0, min=0.1, readout_format='.1f'), FloatSlider(value=0.1, continuous_update=False, description='$\\Delta_t$ =', max=0.5, min=0.05, step=0.05), IntSlider(value=50, continuous_update=False, description='# Obs. =', min=1), Checkbox(value=False, description='Observed v. Q(Post)'), Checkbox(value=True, description='Smooth Posterior'))), VBox(children=(Checkbox(value=True, description='Fixed Noise Model'), HBox(children=(IntSlider(value=1000, continuous_update=False, description='Samples $N$', max=10000, min=250, orientation='vertical', step=500), IntSlider(value=5, continuous_update=False, description='Trials', max=25, min=1, orientation='vertical'), FloatSlider(value=0.15, continuous_update=False, description='$\\sigma$', max=1.75, min=0.05, orientation='vertical', step=0.05))))))),), layout=Layout(justify_content='center'), _titles={'0': 'UQ Sandbox'}), Output()), layout=Layout(justify_content='center')),)), HBox(children=(VBox(children=(Accordion(children=(HBox(children=(VBox(children=(FloatRangeSlider(value=(3.0, 6.0), continuous_update=False, description='Param: $\\Lambda \\in$', max=7.0, min=2.0, step=0.5), FloatSlider(value=4.5, continuous_update=False, description='IC: $\\lambda_0$', max=6.0, min=3.0), FloatSlider(value=0.2, continuous_update=False, description='$t_0$ =', max=2.0, min=0.1, readout_format='.1f'), FloatSlider(value=0.1, continuous_update=False, description='$\\Delta_t$ =', max=0.5, min=0.05, step=0.05), IntSlider(value=50, continuous_update=False, description='# Obs. =', min=1), Checkbox(value=False, description='Observed v. Q(Post)'), Checkbox(value=True, description='Smooth Posterior'))), VBox(children=(Checkbox(value=True, description='Fixed Noise Model'), HBox(children=(IntSlider(value=1000, continuous_update=False, description='Samples $N$', max=10000, min=250, orientation='vertical', step=500), IntSlider(value=5, continuous_update=False, description='Trials', max=25, min=1, orientation='vertical'), FloatSlider(value=0.15, continuous_update=False, description='$\\sigma$', max=1.75, min=0.05, orientation='vertical', step=0.05))))))),), layout=Layout(justify_content='center'), _titles={'0': 'UQ Sandbox'}), Output()), layout=Layout(justify_content='center')),)), HBox(children=(VBox(children=(Accordion(children=(HBox(children=(VBox(children=(FloatRangeSlider(value=(3.0, 6.0), continuous_update=False, description='Param: $\\Lambda \\in$', max=7.0, min=2.0, step=0.5), FloatSlider(value=4.5, continuous_update=False, description='IC: $\\lambda_0$', max=6.0, min=3.0), FloatSlider(value=0.3, continuous_update=False, description='$t_0$ =', max=2.0, min=0.1, readout_format='.1f'), FloatSlider(value=0.1, continuous_update=False, description='$\\Delta_t$ =', max=0.5, min=0.05, step=0.05), IntSlider(value=50, continuous_update=False, description='# Obs. =', min=1), Checkbox(value=False, description='Observed v. Q(Post)'), Checkbox(value=True, description='Smooth Posterior'))), VBox(children=(Checkbox(value=True, description='Fixed Noise Model'), HBox(children=(IntSlider(value=1000, continuous_update=False, description='Samples $N$', max=10000, min=250, orientation='vertical', step=500), IntSlider(value=5, continuous_update=False, description='Trials', max=25, min=1, orientation='vertical'), FloatSlider(value=0.15, continuous_update=False, description='$\\sigma$', max=1.75, min=0.05, orientation='vertical', step=0.05))))))),), layout=Layout(justify_content='center'), _titles={'0': 'UQ Sandbox'}), Output()), layout=Layout(justify_content='center')),)), HBox(children=(VBox(children=(Accordion(children=(HBox(children=(VBox(children=(

### If you get to measure early, more measurements provide little gain

In [6]:
multiplier = 5
num_obs = multiplier*np.array([1,2,3,4,5,10]).astype('int')

T2, K2 = make_tabulated_sandbox(num_experiments)
for k in range(num_experiments):
    K2[k]['num_observations'].value = num_obs[k]
    K2[k]['num_trials'].value = 3

prop = 'fixed_noise'
interact(set_prop, K = fixed(K2), prop=fixed(prop), 
                val = Checkbox(value=True, description = 'Fix Noise') );
prop = 'smooth_post'
interact(set_prop, K = fixed(K2), prop=fixed(prop), 
                val = Checkbox(value=True, description = 'Smooth Post') );

prop = 't_0'
interact_manual(set_prop, K = fixed(K2), prop=fixed(prop), 
                val = FloatSlider(value=0.1, min=0.1, max=2, step=0.1, description='$t_0$ =') );
prop = 'num_samples'
interact_manual(set_prop, K = fixed(K2), prop=fixed(prop), 
                val = IntSlider(value=1000, min=int(5E2), max=int(1E4), step=250, description='Samp. $N$ =') );
prop = 'Delta_t'
interact_manual(set_prop, K = fixed(K2), prop=fixed(prop), 
                val = FloatSlider(value=0.1, min=0.05, max=0.5, step=0.05, description='$\Delta_t$ =') );
prop = 'sd'
interact_manual(set_prop, K = fixed(K2), prop=fixed(prop), 
                val = FloatSlider(value=0.1, min=0.05, max=0.25, step=0.01, description='Constant $\sigma$:' ));
T2

interactive(children=(Checkbox(value=True, description='Fix Noise'), Output()), _dom_classes=('widget-interact',))

interactive(children=(Checkbox(value=True, description='Smooth Post'), Output()), _dom_classes=('widget-interact',))

interactive(children=(FloatSlider(value=0.1, description='$t_0$ =', max=2.0, min=0.1), Button(description='Run Interact', style=ButtonStyle()), Output()), _dom_classes=('widget-interact',))

interactive(children=(IntSlider(value=1000, description='Samp. $N$ =', max=10000, min=500, step=250), Button(description='Run Interact', style=ButtonStyle()), Output()), _dom_classes=('widget-interact',))

interactive(children=(FloatSlider(value=0.1, description='$\\Delta_t$ =', max=0.5, min=0.05, step=0.05), Button(description='Run Interact', style=ButtonStyle()), Output()), _dom_classes=('widget-interact',))

interactive(children=(FloatSlider(value=0.1, description='Constant $\\sigma$:', max=0.25, min=0.05, step=0.01), Button(description='Run Interact', style=ButtonStyle()), Output()), _dom_classes=('widget-interact',))

Tab(children=(HBox(children=(VBox(children=(Accordion(children=(HBox(children=(VBox(children=(FloatRangeSlider(value=(3.0, 6.0), continuous_update=False, description='Param: $\\Lambda \\in$', max=7.0, min=2.0, step=0.5), FloatSlider(value=4.5, continuous_update=False, description='IC: $\\lambda_0$', max=6.0, min=3.0), FloatSlider(value=0.5, continuous_update=False, description='$t_0$ =', max=2.0, min=0.1, readout_format='.1f'), FloatSlider(value=0.1, continuous_update=False, description='$\\Delta_t$ =', max=0.5, min=0.05, step=0.05), IntSlider(value=5, continuous_update=False, description='# Obs. =', min=1), Checkbox(value=False, description='Observed v. Q(Post)'), Checkbox(value=True, description='Smooth Posterior'))), VBox(children=(Checkbox(value=True, description='Fixed Noise Model'), HBox(children=(IntSlider(value=1000, continuous_update=False, description='Samples $N$', max=10000, min=250, orientation='vertical', step=500), IntSlider(value=3, continuous_update=False, description='Trials', max=25, min=1, orientation='vertical'), FloatSlider(value=0.25, continuous_update=False, description='$\\sigma$', max=1.75, min=0.05, orientation='vertical', step=0.05))))))),), layout=Layout(justify_content='center'), _titles={'0': 'UQ Sandbox'}), Output()), layout=Layout(justify_content='center')),)), HBox(children=(VBox(children=(Accordion(children=(HBox(children=(VBox(children=(FloatRangeSlider(value=(3.0, 6.0), continuous_update=False, description='Param: $\\Lambda \\in$', max=7.0, min=2.0, step=0.5), FloatSlider(value=4.5, continuous_update=False, description='IC: $\\lambda_0$', max=6.0, min=3.0), FloatSlider(value=0.5, continuous_update=False, description='$t_0$ =', max=2.0, min=0.1, readout_format='.1f'), FloatSlider(value=0.1, continuous_update=False, description='$\\Delta_t$ =', max=0.5, min=0.05, step=0.05), IntSlider(value=10, continuous_update=False, description='# Obs. =', min=1), Checkbox(value=False, description='Observed v. Q(Post)'), Checkbox(value=True, description='Smooth Posterior'))), VBox(children=(Checkbox(value=True, description='Fixed Noise Model'), HBox(children=(IntSlider(value=1000, continuous_update=False, description='Samples $N$', max=10000, min=250, orientation='vertical', step=500), IntSlider(value=3, continuous_update=False, description='Trials', max=25, min=1, orientation='vertical'), FloatSlider(value=0.25, continuous_update=False, description='$\\sigma$', max=1.75, min=0.05, orientation='vertical', step=0.05))))))),), layout=Layout(justify_content='center'), _titles={'0': 'UQ Sandbox'}), Output()), layout=Layout(justify_content='center')),)), HBox(children=(VBox(children=(Accordion(children=(HBox(children=(VBox(children=(FloatRangeSlider(value=(3.0, 6.0), continuous_update=False, description='Param: $\\Lambda \\in$', max=7.0, min=2.0, step=0.5), FloatSlider(value=4.5, continuous_update=False, description='IC: $\\lambda_0$', max=6.0, min=3.0), FloatSlider(value=0.5, continuous_update=False, description='$t_0$ =', max=2.0, min=0.1, readout_format='.1f'), FloatSlider(value=0.1, continuous_update=False, description='$\\Delta_t$ =', max=0.5, min=0.05, step=0.05), IntSlider(value=15, continuous_update=False, description='# Obs. =', min=1), Checkbox(value=False, description='Observed v. Q(Post)'), Checkbox(value=True, description='Smooth Posterior'))), VBox(children=(Checkbox(value=True, description='Fixed Noise Model'), HBox(children=(IntSlider(value=1000, continuous_update=False, description='Samples $N$', max=10000, min=250, orientation='vertical', step=500), IntSlider(value=3, continuous_update=False, description='Trials', max=25, min=1, orientation='vertical'), FloatSlider(value=0.25, continuous_update=False, description='$\\sigma$', max=1.75, min=0.05, orientation='vertical', step=0.05))))))),), layout=Layout(justify_content='center'), _titles={'0': 'UQ Sandbox'}), Output()), layout=Layout(justify_content='center')),)), HBox(children=(VBox(children=(Accordion(children=(HBox(children=(VBox(children=(F

## Manual Interactive Mode (no tabs)
Once you find some parameters you like, you can run multiple instances of it below to get a sense of the variations in the solutions:



In [7]:
interact_manual(sandbox, 
        num_samples = IntSlider(value=2500, 
            min=int(5E2), max=int(5E4), step=500, description='Samp. $N$ ='), 
        lam_bound = FloatRangeSlider(value=[3.0, 6.0], 
            min=2.0, max = 7.0, step=0.5, description='Param $\Lambda \in$'),
        lam_0 = FloatSlider(value=3.5, 
            min=2.0, max=7.0, step=0.1, description='IC: $\lambda_0$ ='), 
        t_0 = FloatSlider(value=0.5, 
            min=0.1, max=2, step=0.1, description='$t_0$ ='),
        Delta_t = FloatSlider(value=0.1, 
            min=0.05, max=0.5, step=0.05, description='$\Delta_t$ ='),
        num_observations = IntSlider(value=50, 
            min=1, max=100, description='# of Obs. ='), 
        sd = FloatSlider(value=0.1, 
            min=0.05, max=0.25, step=0.01, description='Constant $\sigma$:'), 
        fixed_noise=fixed(True), 
        compare = fixed(False),
        smooth_post = fixed(False)); 

interactive(children=(IntSlider(value=2500, description='Samp. $N$ =', max=50000, min=500, step=500), FloatRangeSlider(value=(3.0, 6.0), description='Param $\\Lambda \\in$', max=7.0, min=2.0, step=0.5), FloatSlider(value=3.5, description='IC: $\\lambda_0$ =', max=7.0, min=2.0), FloatSlider(value=0.5, description='$t_0$ =', max=2.0, min=0.1), FloatSlider(value=0.1, description='$\\Delta_t$ =', max=0.5, min=0.05, step=0.05), IntSlider(value=50, description='# of Obs. =', min=1), FloatSlider(value=0.1, description='Constant $\\sigma$:', max=0.25, min=0.05, step=0.01), IntSlider(value=1, description='num_trials', max=3, min=-1), Button(description='Run Interact', style=ButtonStyle()), Output()), _dom_classes=('widget-interact',))

---

### Suggestions

- Increase $N$ and watch the Pushforward of the Prior change/converge.
- If you broaden the standard deviation $\sigma$, we suggest to also broaden the bound on the parameter space $\Lambda$ in order to avoid voilating the predictability assumption.
- Notice the relationship between the bound on the interval we are inverting for the Mean Squared Error and the support of the posterior.
- The same happens as you increase $\sigma$.
- Change the initial condition $\lambda_0$ and watch the posterior distribution follow the slider.



- Fix the number of observations to 1 and change the interval over which the observation is being made (with $K=0$, the observation occurs only at $T_0$). Notice the diminishing returns as you wait to make your measurement. 
- Fix some interval and change the number of observations made during this time period.
- Fix a number of observations (several) and fix $T_0$ while changing $T$ to observe another example of diminshing returns.

### Observations

- Entropy barely changes as $\lambda_0$ moves around. Increases a bit near boundary of $\Lambda$ (likely due to predictability assumption being violated)
- For a wide time measurement window, entropy increases with the number of observations (d.o.f.)
- Widening $\Lambda$ decreases entropy, obviously enlarges $\mathcal{D}$, support of $P_\mathcal{D}$.
- If you narrow the window, the entropy decreases.
- As the window slides earlier in time, the entropy decreases.
- Higher MSE threshold ($\epsilon$, support of observed density) means higher entropy.
- Higher variance means higher entropy. We might run a suite of $\sigma$s MADS-style to study the robustness of a design. 
    - perhaps if we try to minimize entropy (maximize information gain), we look for designs that are less sensitive to the choice of $\sigma$s, which would **correspond to an experimental design that is robust to measurement uncertainty.**
- Increasing the number of samples $N$ increases entropy quite a bit. Would like to figure out a way to control for this? _Is it even right to be using `scipy.stats.entropy`?_